<a href="https://colab.research.google.com/github/witoriamanuely/aprendizagem_de_maquina_2022.2/blob/master/regularizacao/regularized.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import math
import matplotlib.pyplot as plt
from scipy.stats import skew
from scipy.stats.stats import pearsonr

from sklearn import preprocessing
from sklearn.linear_model import Ridge, RidgeCV, ElasticNet, LassoCV, LassoLarsCV
from sklearn.model_selection import cross_val_score

from sklearn.datasets import make_blobs
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split

knn = KNeighborsRegressor(n_neighbors=7)
%config InlineBackend.figure_format = 'png' #set 'png' here when working on notebook
%matplotlib inline

In [4]:
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv ('/content/drive/MyDrive/mestrado/2022.2/aprendizagem_de_maquina_2022.2/2olab/eleicoes_2006_a_2010.csv')
df_2014 = pd.read_csv ('/content/drive/MyDrive/mestrado/2022.2/aprendizagem_de_maquina_2022.2/2olab/eleicoes_2014.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 2.1 - Analisar as distribuições das variáveis para ver se estão enviesadas e precisam de correção; tratamento de valores ausentes, variáveis categóricas e normalização, quando for o caso.
**Abaixo temos a retiradas dos NaN encontrados no meio dos dados, também há a separação dos dados de treino para os dados do ano de 2006 e os dados de teste para os dados do ano de 2010**

In [5]:


# Concatena os dados entre dos anos de 2006+2010 e 2014
df_all = pd.concat([df, df_2014], ignore_index=True)

#Retirando variáveis categoricas
df_all = df_all.drop(columns=["sequencial_candidato", "nome", "cargo"])

# Normaliza todos os valores
df_all = df_all.fillna(df_all.mean().round(1))
anos = df_all["ano"]
index = df_all.dtypes[df_all.dtypes != "object"].index

df_all[index] = np.log1p(df_all[index])
df_all = pd.get_dummies(df_all)

scaler = preprocessing.MinMaxScaler()
df_all[index] = scaler.fit_transform(df_all[index])
df_all["ano"] = anos


# Separa os dados de treino e teste
x_2010 = df_all[df_all["ano"] == 2010]
x_2006 = df_all[df_all["ano"] == 2006]
x_train = pd.concat([x_2010, x_2006], ignore_index=True)
x_test = df_all[df_all["ano"] == 2014]

# Separa os valores que devem ser feito a predição
y_train = x_train.votos
y_test = x_test.votos

# Remove valores dos dados de treino e validação
x_train = x_train.drop(columns=["ano","votos"])
x_test = x_test.drop(columns=["ano","votos"])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  


# 2.2 Construir modelos de regressão com (ridge e lasso) 

In [6]:
#Modelos abaixo:
model_lasso = LassoCV(cv=10).fit(x_train, y_train)
model_ridge = Ridge(alpha=1).fit(x_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:644: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.019844782505980874, tolerance: 0.012582000650803364
  positive,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:644: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.01959224869234788, tolerance: 0.01275325219683901
  positive,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:644: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.016532360674752056, tolerance: 0.012654523221707134
  positive,


Abaixo temos a acuracia de cada modelo:

In [7]:
print('Score de treino em Lasso', model_lasso.score(x_train, y_train))
print('Score de Validação em Lasso', model_lasso.score(x_test, y_test))

print('Score de treino em Ridge', model_ridge.score(x_train, y_train))
print('Score de Validação em Ridge', model_ridge.score(x_test, y_test))

Score de treino em Lasso 0.7727459383277528
Score de Validação em Lasso -0.19053431083605776
Score de treino em Ridge 0.7789868208122119
Score de Validação em Ridge -0.2788891372527771


#2.3 considerar outros modelos ainda não vistos em sala de sua escolha (e.g. SVR, Regression Trees, KNN e Random Florests).

Nesse caso escolhi o KNN como outro modelo.

In [ ]:
model_knn = knn.fit(x_train, y_train)
print('Score de treino em KNN', model_knn.score(x_train, y_train))
print('Score de Validação em KNN', model_knn.score(x_test, y_test))

# 2.5 Plotar os resíduos versus predições e analisar se esses plots representam bons indícios da adequabilidade dos modelos a esse problema.

##Abaixo temos a comparação da Predição e dos valores reais no modelo Lasso

In [ ]:
lasso_results = model_ridge.predict(x_test)
plt.subplots(figsize=(18, 10))
plt.subplot(211)
plt.title("Predição - Lasso")
plt.plot(range(len(lasso_results)), lasso_results)

plt.subplot(212)
plt.title("Valores Reais")
plt.plot(range(len(y_test)), y_test)

In [ ]:
coef = pd.Series(model_lasso.coef_, index = x_train.columns)
print("Lasso retirou " + str(sum(coef != 0)) + " variáveis e eliminou outras " +  str(sum(coef == 0)) + " variáveis")
imp_coef = pd.concat([coef.sort_values().head(10),
                     coef.sort_values().tail(10)])
matplotlib.rcParams['figure.figsize'] = (8.0, 10.0)
imp_coef.plot(kind = "barh")
plt.title("Coeficientes no modelo Lasso")

In [ ]:
matplotlib.rcParams['figure.figsize'] = (6.0, 6.0)
preds = pd.DataFrame({"preds":model_lasso.predict(x_train), "true":y_train})
preds["residuals"] = preds["true"] - preds["preds"]
preds.plot(x = "preds", y = "residuals",kind = "scatter")

##Abaixo temos a comparação da Predição e dos valores reais no modelo Ridge

In [ ]:
ridge_results = model_ridge.predict(x_test)
plt.subplots(figsize=(18, 10))
plt.subplot(211)
plt.title("Predição - Ridge")
plt.plot(range(len(ridge_results)), ridge_results)

plt.subplot(212)
plt.title("Valores Reais")
plt.plot(range(len(y_test)), y_test)

In [ ]:
coef = pd.Series(model_ridge.coef_, index = x_train.columns)
print("Ridge retirou " + str(sum(coef != 0)) + " variáveis e eliminou outras " +  str(sum(coef == 0)) + " variáveis")
imp_coef = pd.concat([coef.sort_values().head(10),
                     coef.sort_values().tail(10)])
matplotlib.rcParams['figure.figsize'] = (8.0, 10.0)
imp_coef.plot(kind = "barh")
plt.title("Coeficientes no modelo Ridge")

In [ ]:
matplotlib.rcParams['figure.figsize'] = (6.0, 6.0)
preds = pd.DataFrame({"preds":model_ridge.predict(x_train), "true":y_train})
preds["residuals"] = preds["true"] - preds["preds"]
preds.plot(x = "preds", y = "residuals",kind = "scatter")

##Abaixo temos a comparação da Predição e dos valores reais no modelo KNN

In [ ]:
knn_results = model_knn.predict(x_test)
plt.subplots(figsize=(18, 10))
plt.subplot(211)
plt.title("Predição - KNN")
plt.plot(range(len(knn_results)), knn_results)

plt.subplot(212)
plt.title("Valores Reais")
plt.plot(range(len(y_test)), y_test)

In [ ]:
matplotlib.rcParams['figure.figsize'] = (6.0, 6.0)
preds = pd.DataFrame({"preds":model_knn.predict(x_train), "true":y_train})
preds["residuals"] = preds["true"] - preds["preds"]
preds.plot(x = "preds", y = "residuals",kind = "scatter")

#RESPOSTA - 2.5: Tanto o ranking de Lasso como Ridge são semelhantes, porém  Lasso eliminou mais coeficientes que Ridge, enquanto Ridge escolheu mais coeficientes que o Lasso. Não teve como plotar o ranking do KNN devido a lib do KNeighborsRegressor não possuir o atributo coef_. Foi possível ver que os modelos possuem dados semelhantes uns aos outros e também próximos aos dados de teste. 


# 5 Dentre os modelos avaliados, qual foi o que deu o melhor resultado nos dados de 2014 em termos de RMSE? Justifique bem sua resposta.

In [ ]:
def rmse_cv(model):
  rmse= np.sqrt(-cross_val_score(model, x_train, y_train, scoring="neg_mean_squared_error"))
  return(rmse)
#print("Lasso RMSE" + rmse_cv(model_lasso))
#print("Ridge RMSE" + rmse_cv(model_ridge))
#print("KNN RMSE"+ rmse_cv(model_knn))

Abaixo temos um plot do resultado final de treino e teste, com isso pode-se concluir que embora.... 

In [ ]:
import xgboost as xgb
dtrain = xgb.DMatrix(x_train, label = y_train)
dtest = xgb.DMatrix(x_test)

params = {"max_depth":2, "eta":0.1}
model = xgb.cv(params, dtrain,  num_boost_round=500, early_stopping_rounds=100)
model.loc[30:,["test-rmse-mean", "train-rmse-mean"]].plot()

In [ ]:
%%capture
!pip install nbconvert
!jupyter nbconvert --to html Previsao_Eleicao_Deputados.ipynb